In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# Term Simplification via  Rewriting

In [ ]:
import string

In [ ]:
def is_variable(s):
    return isinstance(s, str) and s != '' and s[0] in string.ascii_uppercase

In [ ]:
def match(pattern, term, Substitution={}):
    if is_variable(pattern):
        V = pattern
        if V in Substitution:
            return match(Substitution[V], term, Substitution)
        else:
            Substitution[V] = term
            return True
    if isinstance(pattern, str) or isinstance(pattern, int):
        return pattern == term
    if isinstance(term, str) or isinstance(term, int):
        return False
    if len(pattern) != len(term):
        return False
    if pattern[0] != term[0]:
        return False
    n = len(pattern)
    for i in range(n):
        if not match(pattern[i], term[i], Substitution):
            return False
    return True

In [ ]:
def apply(term, Substitution):
    if is_variable(term):
        V = term
        if V in Substitution:
            return Substitution[V]
        else:
            return V
    if isinstance(term, str) or isinstance(term, int):
        return term
    return tuple(apply(arg, Substitution) for arg in term)

In [ ]:
def rewrite(term, rule):
    lhs, rhs = rule
    Substitution = {}
    if match(lhs, term, Substitution):
        return True, apply(rhs, Substitution)
    else:
        return False, term

In [ ]:
def simplify_once(term, Rules):
    if isinstance(term, str) or isinstance(term, int):
        return term
    for rule in Rules:
        flag, simple = rewrite(term, rule)
        if flag:
            return simple
    return tuple(simplify_once(arg, Rules) for arg in term)

In [ ]:
def simplify(t, Rules):
    while True:
        old_t = t
        t     = simplify_once(t, Rules)
        if t == old_t:
            return t

In [ ]:
Rules = {
    (('+', 'R', 0), 'R'),
    (('+', 0, 'R'), 'R'),
    (('+', 'R', 'R'), 'R'),
    (('+', '', ('*', 'R')), ('*', 'R')),
    (('+', ('*', 'R'), ''), ('*', 'R')),
    (('+', '', ('&', 'R', ('*', 'R'))), ('*', 'R')),
    (('+', '', ('&', ('*', 'R'), 'R')), ('*', 'R')),
    (('+', ('&', 'R', ('*', 'R')), ''), ('*', 'R')),
    (('+', ('&', ('*', 'R'), 'R'), ''), ('*', 'R')),
    (('+', 'S', ('&', 'S', 'T')), ('&', 'S', ('+', '', 'T'))),
    (('+', 'S', ('&', 'T', 'S')), ('&', ('+', '', 'T'), 'S')),
    (('&', 0, 'R'), 0),
    (('&', 'R', 0), 0),
    (('&', '', 'R'), 'R'),
    (('&', '', 'R'), 'R'),
    (('&', ('+', '', 'R'), ('*', 'R')), ('*', 'R')),
    (('&', ('+', 'R', ''), ('*', 'R')), ('*', 'R')),
    (('&', ('*', 'R'), ('+', 'R', '')), ('*', 'R')),
    (('&', ('*', 'R'), ('+', '', 'R')), ('*', 'R')),
    (('*', 0), ''),
    (('*', ''), ''),
    (('*', ('+', '', 'R')), ('*', 'R')),
    (('*', ('+', 'R', '')), ('*', 'R')),
    (('+', 'R', ('+', 'S', 'T')), ('+', ('+', 'R', 'S'), 'T')),
    (('&', 'R', ('&', 'S', 'T')), ('&', ('&', 'R', 'S'), 'T')),
    (('&', ('&', 'R', ('*', 'S')), ('+', '', 'S')), ('&', 'R', ('*', 'S')))
}